In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np 
from scipy import stats
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
import catboost as catb
from scipy.stats import f_oneway
import seaborn as sns
from matplotlib import pyplot as plt

In [3]:
TRAIN_DATASET_1 = 'country_indicators.csv'
TRAIN_DATASET_2 = 'country_codes.csv'
TRAIN_DATASET_3 = 'pirate_attacks.csv'
TRAIN_DATASET_PATH = 'F:/Lesson/course_project_train.csv'
df = pd.read_csv(TRAIN_DATASET_PATH)
df.head(10)

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0
3,Own Home,805068.0,6 years,0.0,8.0,22.5,147400.0,1.0,NaN,1.0,debt consolidation,Short Term,121396.0,95855.0,11338.0,694.0,0
4,Rent,776264.0,8 years,0.0,13.0,13.6,385836.0,1.0,NaN,0.0,debt consolidation,Short Term,125840.0,93309.0,7180.0,719.0,0
5,Rent,NaN,7 years,0.0,12.0,14.6,366784.0,0.0,NaN,0.0,other,Long Term,337304.0,165680.0,18692.0,NaN,1
6,Home Mortgage,1511108.0,10+ years,0.0,9.0,20.3,388124.0,0.0,73.0,0.0,home improvements,Short Term,99999999.0,51623.0,2317.0,745.0,0
7,Rent,1040060.0,10+ years,0.0,13.0,12.0,330374.0,0.0,18.0,0.0,other,Short Term,250888.0,89015.0,19761.0,705.0,1
8,Home Mortgage,NaN,5 years,0.0,17.0,15.7,0.0,1.0,NaN,1.0,home improvements,Short Term,129734.0,19.0,17.0,NaN,0
9,Home Mortgage,NaN,1 year,0.0,10.0,24.6,511302.0,0.0,6.0,0.0,debt consolidation,Long Term,572880.0,205333.0,17613.0,NaN,1


In [4]:
# берем среднее значение для NaN значений в 'Annual Income'
df['Annual Income'].fillna(df['Annual Income'].median(), inplace=True)

# Приравниваем значения NaN к нулю, и делим все значения в столбце 'Months since last delinquent' на 100
# таким образом все значения в стольбце распределяются по возрастающей стремящеся к нулю.
df.fillna(0, inplace=True)
df['Months since last delinquent'] = df['Months since last delinquent']/100

# приводим обьекты к цифровым значениям
le = LabelEncoder()
df['Home Ownership'] = le.fit_transform(df['Home Ownership'])

df['Purpose'] = le.fit_transform(df['Purpose'])

df['Term'] = le.fit_transform(df['Term'])

# Заменить значения на цифры
replace_dict = {'< 1 year':0,'1 year':1, '2 years':2, '3 years':3, '4 years':4, '5 years':5, '6 years':6, '7 years':7, '8 years':8, '9 years':9, '10+ years':10}

df['Years in current job'].replace(replace_dict, inplace=True)

df['Years in current job'].astype(int)



0        0
1       10
2        8
3        6
4        8
        ..
7495     0
7496     1
7497     6
7498     0
7499     4
Name: Years in current job, Length: 7500, dtype: int32

In [5]:
df.head(10)

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,2,482087.0,0,0.0,11.0,26.3,685960.0,1.0,0.00,1.0,3,1,99999999.0,47386.0,7914.0,749.0,0
1,2,1025487.0,10,0.0,15.0,15.3,1181730.0,0.0,0.00,0.0,3,0,264968.0,394972.0,18373.0,737.0,1
2,1,751412.0,8,0.0,11.0,35.0,1182434.0,0.0,0.00,0.0,3,1,99999999.0,308389.0,13651.0,742.0,0
3,2,805068.0,6,0.0,8.0,22.5,147400.0,1.0,0.00,1.0,3,1,121396.0,95855.0,11338.0,694.0,0
4,3,776264.0,8,0.0,13.0,13.6,385836.0,1.0,0.00,0.0,3,1,125840.0,93309.0,7180.0,719.0,0
5,3,1168386.0,7,0.0,12.0,14.6,366784.0,0.0,0.00,0.0,9,0,337304.0,165680.0,18692.0,0.0,1
6,1,1511108.0,10,0.0,9.0,20.3,388124.0,0.0,0.73,0.0,5,1,99999999.0,51623.0,2317.0,745.0,0
7,3,1040060.0,10,0.0,13.0,12.0,330374.0,0.0,0.18,0.0,9,1,250888.0,89015.0,19761.0,705.0,1
8,1,1168386.0,5,0.0,17.0,15.7,0.0,1.0,0.00,1.0,5,1,129734.0,19.0,17.0,0.0,0
9,1,1168386.0,1,0.0,10.0,24.6,511302.0,0.0,0.06,0.0,3,0,572880.0,205333.0,17613.0,0.0,1


In [6]:
# df[['Home Ownership','Annual Income','Years in current job',
#     'Tax Liens','Number of Open Accounts','Years of Credit History',
#     'Maximum Open Credit','Number of Credit Problems',
#     'Months since last delinquent','Bankruptcies',
#     'Purpose','Term','Current Loan Amount','Current Credit Balance',
#     'Monthly Debt','Credit Score','Credit Default']].corr()

In [7]:
df[['Annual Income','Maximum Open Credit','Credit Default']].corr()

,Annual Income,Maximum Open Credit,Credit Default
Annual Income,1.000000,0.019311,-0.095203
Maximum Open Credit,0.019311,1.000000,-0.014275
Credit Default,-0.095203,-0.014275,1.000000


In [8]:
# Выберем данные только с погашенными кредитами
df = df.loc[df['Credit Default'] == 1]
# Разделим базу по признаку владения жильем
Home_Mortgage = df.loc[df['Home Ownership'] == 1]
Own_Home = df.loc[df['Home Ownership'] == 2]
Rent = df.loc[df['Home Ownership'] == 3]

Home_Mortgage.reset_index(inplace=True)
Own_Home.reset_index(inplace=True)
Rent.reset_index(inplace=True)


In [9]:
Home_Mortgage

,index,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,9,1,1168386.0,1,0.0,10.0,24.6,511302.0,0.0,0.06,0.0,3,0,572880.0,205333.0,17613.0,0.0,1
1,17,1,3361898.0,3,0.0,14.0,28.5,1042096.0,0.0,0.00,0.0,3,0,498872.0,689396.0,52249.0,644.0,1
2,18,1,1401744.0,2,0.0,9.0,29.0,387222.0,0.0,0.40,0.0,3,0,553586.0,201989.0,13434.0,695.0,1
3,21,1,1047394.0,6,0.0,7.0,34.4,401104.0,0.0,0.45,0.0,3,1,324852.0,183597.0,7786.0,719.0,1
4,32,1,1526840.0,10,0.0,19.0,25.0,1303082.0,0.0,0.14,0.0,3,1,265188.0,495178.0,28247.0,726.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,7471,1,1534839.0,4,0.0,12.0,19.0,427768.0,0.0,0.34,0.0,3,0,526570.0,338409.0,30313.0,718.0,1
912,7472,1,1043537.0,5,0.0,9.0,14.7,726902.0,0.0,0.00,0.0,3,0,539902.0,315780.0,14262.0,726.0,1
913,7487,1,1542952.0,10,0.0,7.0,12.5,232364.0,0.0,0.59,0.0,3,0,267982.0,198873.0,15430.0,695.0,1
914,7489,1,1394942.0,10,0.0,15.0,27.2,1441396.0,0.0,0.35,0.0,3,1,753764.0,496698.0,16972.0,742.0,1


In [10]:
Own_Home

,index,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,1,2,1025487.0,10,0.0,15.0,15.3,1181730.0,0.0,0.00,0.0,3,0,264968.0,394972.0,18373.0,737.0,1
1,23,2,1168386.0,10,0.0,17.0,14.8,700040.0,0.0,0.17,0.0,3,1,174878.0,168074.0,17132.0,0.0,1
2,42,2,1168386.0,4,0.0,22.0,10.9,625042.0,0.0,0.28,0.0,3,0,326810.0,300675.0,13844.0,0.0,1
3,143,2,1168386.0,10,0.0,6.0,20.4,249392.0,0.0,0.00,0.0,3,1,331056.0,152285.0,10321.0,0.0,1
4,174,2,708491.0,1,0.0,16.0,19.4,559020.0,0.0,0.21,0.0,3,1,326524.0,438862.0,20960.0,709.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,7351,2,664012.0,9,0.0,8.0,16.9,232738.0,1.0,0.38,1.0,3,1,219670.0,52668.0,6585.0,742.0,1
188,7379,2,290415.0,0,0.0,16.0,12.5,116644.0,0.0,0.00,0.0,11,1,71170.0,55499.0,2953.0,7160.0,1
189,7428,2,1168386.0,0,0.0,9.0,18.9,234212.0,0.0,0.00,0.0,3,1,133320.0,114076.0,3306.0,0.0,1
190,7436,2,1070080.0,6,0.0,17.0,19.0,405702.0,0.0,0.57,0.0,3,0,337348.0,234061.0,24701.0,645.0,1


In [11]:
Rent

,index,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,5,3,1168386.0,7,0.0,12.0,14.6,366784.0,0.0,0.00,0.0,9,0,337304.0,165680.0,18692.0,0.0,1
1,7,3,1040060.0,10,0.0,13.0,12.0,330374.0,0.0,0.18,0.0,9,1,250888.0,89015.0,19761.0,705.0,1
2,13,3,2066364.0,10,0.0,11.0,17.7,1654026.0,0.0,0.00,0.0,3,0,549780.0,1037077.0,21869.0,728.0,1
3,16,3,1168386.0,7,0.0,6.0,17.8,899492.0,0.0,0.00,0.0,3,0,313038.0,538346.0,18844.0,0.0,1
4,19,3,1020072.0,2,0.0,6.0,12.5,199078.0,0.0,0.00,0.0,3,1,309672.0,120517.0,14451.0,712.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,7460,3,769462.0,5,0.0,8.0,17.0,313786.0,0.0,0.80,0.0,3,1,109186.0,45524.0,4585.0,7470.0,1
999,7477,3,1995000.0,3,0.0,16.0,21.0,816618.0,0.0,0.49,0.0,1,1,528000.0,29621.0,9559.0,750.0,1
1000,7483,3,567435.0,8,0.0,11.0,17.1,365750.0,0.0,0.08,0.0,3,0,328526.0,181944.0,6431.0,658.0,1
1001,7485,3,2687664.0,0,0.0,14.0,16.7,429154.0,0.0,0.77,0.0,3,1,466796.0,278711.0,44122.0,736.0,1


Среди людей, берущих кредиты, выделены три группы по жилью: собственное жильё, жилье в ипотеке, съемное. Имеются данные по взятым кредитам этими группами.

Требуется проверить гипотезу у равенстве среднего кредита по каждой группе. Для проверки этой гипотезы воспользуемся однофакторным дисперсионным анализом.

Выборочные средние по каждой группе:

In [12]:
X1 = Home_Mortgage['Maximum Open Credit']
X2 = Own_Home['Maximum Open Credit']
X3 = Rent['Maximum Open Credit']

n1 = X1.shape[0]
n2 = X2.shape[0]
n3 = X3.shape[0]
X1_mean = X1.mean()
X2_mean = X2.mean()
X3_mean = X3.mean()
print(n1,n2,n3)
X1_mean, X2_mean, X3_mean


916 192 1003


(693004.8515283843, 636827.9270833334, 465881.0588235294)

In [13]:
y = np.concatenate([X1, X2, X3])

print(y)

[ 511302. 1042096.  387222. ...  365750.  429154.  821480.]


In [14]:
y_mean = y.mean()
y_mean

579982.0502131691

In [15]:
S2_b = n1 * (X1_mean - y_mean) ** 2 + n2 * (X2_mean - y_mean) ** 2 + n3 * (X3_mean - y_mean) ** 2

S2_w = ((X1 - X1_mean) ** 2).sum() + ((X2 - X2_mean) ** 2).sum() + ((X3 - X3_mean) ** 2).sum()

S2_b, S2_w

(25379657171796.36, 617230555675290.4)

Проверим выполнение равенства  𝑆2=𝑆2𝑏+𝑆2𝑤:

In [16]:
y_mean = round(y_mean, 2)

print(S2_b,S2_w,y,y_mean)

25379657171796.36 617230555675290.4 [ 511302. 1042096.  387222. ...  365750.  429154.  821480.] 579982.05


In [17]:
S2_b + S2_w == ((y - y_mean) ** 2).sum()

True

In [18]:
k = 3
n = n1 + n2 + n3

k1 = k - 1
k2 = n - k

sigma2_b = S2_b / k1
sigma2_w = S2_w / k2

sigma2_b, sigma2_w

(12689828585898.18, 292803868916.1719)

Итак, значение статистики  𝑇 :

In [19]:
T = sigma2_b / sigma2_w
T

43.33900584329781

Зафиксируем уровень значимости  𝛼=0.05 . Для него найдём критическое значение  𝐹𝑐𝑟𝑖𝑡 :

In [20]:
alpha = 0.05

F_crit = stats.f.ppf(1 - alpha, k1, k2)
F_crit

2.9999936209965594

In [21]:
p = 1 - stats.f.cdf(T, k1, k2)
p

1.1102230246251565e-16

Видим, что  𝑇>𝐹𝑐𝑟𝑖𝑡 , поэтому заключаем, что отличие средних кредитных сумм действительно является статистически значимым для выбранных групп владельцев жилья.

In [23]:
f_oneway(X1, X2, X3)

F_onewayResult(statistic=43.33900584329781, pvalue=3.587425705244982e-19)